<a href="https://colab.research.google.com/github/Raj-Harshit23/FinSearch---Option-Pricing-Models/blob/main/BM-BSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git

  Cloning https://github.com/rongardF/tvdatafeed.git to /tmp/pip-req-build-46om5q92
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed.git /tmp/pip-req-build-46om5q92
  Resolved https://github.com/rongardF/tvdatafeed.git to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


In [ ]:
import math
def CRRTree(K,T,S,sig,r,N,PorC):

    dt=T/N;
    dxu=math.exp(sig*math.sqrt(dt));
    dxd=math.exp(-sig*math.sqrt(dt));
    pu=((math.exp(r*dt))-dxd)/(dxu-dxd);
    pd=1-pu;
    disc=math.exp(-r*dt);

    St = [0] * (N+1)
    C = [0] * (N+1)

    St[0]=S*dxd**N;

    for j in range(1, N+1):
        St[j] = St[j-1] * dxu/dxd;

    for j in range(1, N+1):
        if PorC == 'p':
            C[j] = max(K-St[j],0);
        elif PorC == 'c':
            C[j] = max(St[j]-K,0);

    for i in range(N, 0, -1):
        for j in range(0, i):
            C[j] = disc*(pu*C[j+1]+pd*C[j]);

    return C[0]

In [ ]:
import math

def fnor2(x):

    y=0.5*(1+math.erf(x/math.sqrt(2)));

    return y

def BS(t,St,K,T,r,sig,PorC):

    Tmt=T-t;
    ATmt=sig*math.sqrt(Tmt);
    logo=math.log(St/K);
    Ap=(logo+(r+0.5*sig**2)*Tmt)/ATmt;
    An=Ap-ATmt;

    if PorC == 'c':
        p=St*fnor2(Ap)-K*math.exp(-r*Tmt)*fnor2(An);
    elif PorC == 'p':
        p=K*math.exp(-r*Tmt)*fnor2(-An)-St*fnor2(-Ap);

    return p

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval ##used to get the historical data
import random
## top 10 stocks
n50=np.array(['NIFTY','HINDALCO','MARUTI','NESTLEIND', 'ONGC', 'TATAMOTORS','ITC','SUNPHARMA','BHARATIARTL','CIPLA'])
result=pd.DataFrame(columns=['Start', 'Strike', 'Market Premium', 'BS Premium', 'Binomial Premium', 'BS Error', 'Binomial Error'])


for b in range(0,3):
  tv=TvDatafeed()
  data=tv.get_hist(symbol=n50[b],exchange='NSE',interval=Interval.in_daily,n_bars=1100)
  df=pd.DataFrame(data)
  vd=252 ##offset for volatility
  rf=0.0695  ## risk free rate
  df=df.drop(['symbol','high','low','volume'],axis=1)
  df.reset_index(inplace=True)
  df['datetime']=pd.to_datetime(df['datetime'])
  df['day_of_week']=df['datetime'].dt.day_of_week ## if day of week is 3, it is thursday
  df['date']=df['datetime'].dt.day
  df['daily_return']=np.log(df['close']/df['open'])
  #print(df.head())
  act_index=[]
  for i in range(1080-vd):
    if df.loc[i+vd,'date'] > df.loc[i+vd+1,'date']:
      flag=False
      for j in range(5):
        if df.loc[i+vd-j,'day_of_week'] == 3:
          act_index.append(i+vd-j)
          flag=True
          break
      ## incase the given thursadayis a holiday, we would check on the friday. this ensures that we dont skip any month
      ## due to lack of thursday data in the dataframe
      if flag==False:
        for j in range(5):
          if df.loc[i+vd-j,'day_of_week'] == 4:
            act_index.append(i+vd-j)
            flag=True
            break
  #print(len(act_index))
  #print(df.loc[act_index])
  #print(df[410:420])
  ## now calculating for each index/month
  for e in range(39):
    start_index=act_index[e]
    start_price=df.loc[start_index,'open']
    vol=df.loc[start_index-252:start_index,'daily_return']
    vol=vol.to_numpy()
    sigma=vol.std()*np.sqrt(252)
    #print(sigma)

    for num_of_sps in range(-2,3):
      strike_price=start_price+num_of_sps*sigma*start_price*0.5
      bs_premium=BS(0,start_price,strike_price,1.0/12,rf,sigma,'c')
      bt_premium=CRRTree(strike_price,1.0/12,start_price,sigma,rf,500,'c')
      market_premium=df.loc[start_index+1,'open']-strike_price
      result.loc[len(result)] ={
        'Start': start_price,
        'Strike': strike_price,
        'Market Premium': market_premium,
        'BS Premium': bs_premium,
        'Binomial Premium': bt_premium,
        'BS Error': abs(bs_premium - market_premium),
        'Binomial Error': abs(bt_premium - market_premium),
      }
  print(result)